# Flatten gridded data for trend fitting

In [7]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, regionmask

# make a new directory to store the data & results
! mkdir -p flattened; mkdir -p flattened-res

In [8]:
# load daily data
da = xr.open_dataset("era5_tmax_daily_extended_-125--75E_9-37N_su.nc").tmax

# mask land only
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
da = da.where(rm == 0, drop = True)

## Edit the cell below to reflect your data & event definition

In [9]:
# use your event definition to get an annual time series at each grid cell
da = da.rolling(time = 5).mean().resample(time = "YS").max()

# set a file description that will be used to save all the data
fnm_root = "tx5x-era5"

## Run the cell below to flatten the data and save ready for processing

In [10]:
# relabel dates as years
da = da.assign_coords(time = da.time.dt.year).rename(time = "year")

# save the map for easier reconstruction later
da.mean("year").to_netcdf("map-tmplt_"+fnm_root+".nc")

# flatten & convert to data.frame
df = da.stack(xy = ["lat", "lon"]).dropna(dim = "xy", how = "all").to_pandas()

# save data.frame as .csv (split into chunks if really large)
ncols = 2500
if df.shape[1] > ncols:
    for i in range(int(np.ceil(df.shape[1] / ncols))):
        df.iloc[:,slice(i*ncols,(i+1)*ncols)].to_csv("flattened/"+fnm_root+"-flattened_"+str(i+1).rjust(2,"0")+".csv")
else:
    df.to_csv("flattened/"+fnm_root+"-flattened.csv")

**Now open the next workbook (using R) to fit the statistical model over each grid cell**